# Dynamic Hand Gesture Recognition

## Importing

In [1]:
!pip install mediapipe keras tensorflow torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [3]:
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import cv2

# from google.colab.patches import cv2_imshow

import torch.nn as nn
import torch.optim as optim

## Dataset Preprocessing

In [20]:
from huggingface_hub import hf_hub_download

REPO_ID = "Seba213/rgb-dhgr-dataset"
FILENAME = "rgb_blur_split.zip"

dataset = hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset")

c:\Users\User\Desktop\Universita\Magistrale\GeReco\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\Desktop\Universita\Magistrale\GeReco\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\datasets--Seba213--rgb-dhgr-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python 

In [21]:
dataset

'C:\\Users\\User\\.cache\\huggingface\\hub\\datasets--Seba213--rgb-dhgr-dataset\\snapshots\\eac39e89515b334863ea2432c22551b1d6bcd4bf\\rgb_blur_split.zip'

In [24]:
import zipfile
import os

zip_path = r"C:\Users\User\.cache\huggingface\hub\datasets--Seba213--rgb-dhgr-dataset\snapshots\eac39e89515b334863ea2432c22551b1d6bcd4bf\rgb_blur_split.zip"
extract_path = r"C:\Users\User\Desktop\Universita\Magistrale\GeReco"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Estrazione completata!")


Estrazione completata!


In [27]:
import requests

url = "https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task"
save_path = r"C:\Users\User\Downloads\hand_landmarker.task"

response = requests.get(url)
if response.status_code == 200:
    with open(save_path, "wb") as file:
        file.write(response.content)
    print("Download completato!")
else:
    print("Errore nel download:", response.status_code)


Download completato!


In [4]:
mp_drawing = mp.solutions.drawing_utils

In [5]:
TEST_IMAGE_PATH = "/content/rgb_blur_split/train/scroll_right/2021-10-28-103337_8/12.png"

In [6]:
original_img = cv2.imread(TEST_IMAGE_PATH)
cv2_imshow(original_img)

NameError: name 'cv2_imshow' is not defined

In [7]:
MARGIN = 30  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54)  # vibrant green

In [8]:
def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)
    x_coordinates = []
    y_coordinates = []

    # Loop through the detected hands to visualize
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())

        # Get the coordinates of all the keypoints
        height, width, _ = annotated_image.shape
        x_coordinates += [int(landmark.x * width) for landmark in hand_landmarks]
        y_coordinates += [int(landmark.y * height) for landmark in hand_landmarks]

    return annotated_image, x_coordinates, y_coordinates

In [9]:
def crop_hands_region(rgb_image, x_coordinates, y_coordinates):

    height, width, _ = rgb_image.shape

    x_min = max(min(x_coordinates) - MARGIN, 0)
    x_max = min(max(x_coordinates) + MARGIN, width)
    y_min = max(min(y_coordinates) - MARGIN, 0)
    y_max = min(max(y_coordinates) + MARGIN, height)

    return rgb_image[y_min:y_max, x_min:x_max]

In [10]:
def cropping(image_path):
  base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
  options = vision.HandLandmarkerOptions(base_options=base_options,num_hands=2)
  detector = vision.HandLandmarker.create_from_options(options)

  image = mp.Image.create_from_file(image_path)

  detection_result = detector.detect(image)

  annotated_image, x, y = draw_landmarks_on_image(image.numpy_view(), detection_result)

  if not x or not y:
    return None
  else:
    cropped_image = crop_hands_region(image.numpy_view(), x, y)
    return cropped_image

In [35]:
import numpy as np

# Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# Load the input image.
image = mp.Image.create_from_file(TEST_IMAGE_PATH)

# Detect hand landmarks from the input image.
detection_result = detector.detect(image)

# Process the classification result. In this case, visualize it.
annotated_image, x, y = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

# Display the cropped images
# Assuming 'cropped_images' is a list of images:
cropped_image = crop_hands_region(image.numpy_view(), x, y)
cv2_imshow(cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR))

RuntimeError: Image decoding failed (can't fopen): /content/rgb_blur_split/train/scroll_right/2021-10-28-103337_8/12.png

In [11]:
def resize_image(cropped_image):
  image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

  image_resized = cv2.resize(image, (75, 75))

  image_resized = image_resized / 255.0

  image_tensor = tf.convert_to_tensor(image_resized, dtype=tf.float32)

  image_tensor = tf.expand_dims(image_tensor, axis=0)

  return image_tensor

In [12]:
def preprocess_data(image_path):
  cropped_image = cropping(image_path)

  if cropped_image is not None:
    image = resize_image(cropped_image)
    return image
  else:
    return None

In [38]:
import tensorflow as tf

print("Original Image shape:", original_img.shape)

# Load image using OpenCV
image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

# Print shape
print("Cropped Image shape:", image.shape)

# Resize to (75, 75)
image_resized = cv2.resize(image, (75, 75))

# Normalize (optional)
image_resized = image_resized / 255.0

# Convert to tensor
image_tensor = tf.convert_to_tensor(image_resized, dtype=tf.float32)

# Add batch dimension if needed
image_tensor = tf.expand_dims(image_tensor, axis=0)


print("Final tensor shape:", image_tensor.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [13]:
import os
import tensorflow as tf
import numpy as np
from collections import defaultdict

DATASET_PATH = "C:/Users/User/Desktop/Universita/Magistrale/GeReco/rgb_blur_split"

train_dir = os.path.join(DATASET_PATH, "train")

labels = [label for label in os.listdir(train_dir) if not label.startswith('.')]

# Initialize nested dictionary
dataset_dict = defaultdict(lambda: defaultdict(dict))

preprocessed_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

def numerical_sort(file_list):
    return sorted(file_list, key=lambda x: int(os.path.splitext(x)[0]))

# Populate dictionary
for split in ["train", "val", "test"]:
    split_path = os.path.join(DATASET_PATH, split)

    class_labels = [label for label in os.listdir(split_path) if not label.startswith('.')]

    for class_label in class_labels:
        class_path = os.path.join(split_path, class_label)

        examples = [example for example in os.listdir(class_path) if not example.startswith('.')]
        
        # Prendi solo 1/4 degli esempi
        sampled_examples = examples[::4]

        for example in sampled_examples:
            example_path = os.path.join(class_path, example)

            frame_files = numerical_sort(os.listdir(example_path))
            frame_paths = [os.path.join(example_path, f) for f in frame_files if not f.startswith('.')]

            sampled_frames = frame_paths[::4]

            # Store in dictionary
            dataset_dict[split][class_label][example] = sampled_frames

            for frame_path in sampled_frames:
                image_tensor = preprocess_data(frame_path)

                if image_tensor is not None:
                    preprocessed_data[split][class_label][example].append(image_tensor)


In [14]:
import pickle

# Convert defaultdict to a normal dict
normal_dict = {k: dict(v) for k, v in preprocessed_data.items()}

# Save to a pickle file
with open("preprocessed_data.pkl", "wb") as f:
    pickle.dump(normal_dict, f)

In [15]:
# Load the dictionary back
with open("preprocessed_data.pkl", "rb") as f:
    loaded_data = pickle.load(f)

In [16]:
from pprint import pprint
import json

def print_preprocessed_data(preprocessed_data):
    """Prints the preprocessed_data dictionary in a structured format."""
    for split, classes in preprocessed_data.items():
        print(f"\n Split: {split.upper()}")
        for class_label, examples in classes.items():
            print(f"  Class: {class_label}")
            for example, frames in examples.items():
                print(f"    Example: {example}")
                print(f"      Frames Processed ({len(frames)}):")
                for idx, frame in enumerate(frames):
                    print(f"        {idx+1}. Tensor Shape: {frame.shape}")  # Print tensor shape

# Call the function to print the dictionary
print_preprocessed_data(loaded_data)


 Split: TRAIN
  Class: scroll_down
    Example: 2021-10-16-125159_0
      Frames Processed (5):
        1. Tensor Shape: (1, 75, 75, 3)
        2. Tensor Shape: (1, 75, 75, 3)
        3. Tensor Shape: (1, 75, 75, 3)
        4. Tensor Shape: (1, 75, 75, 3)
        5. Tensor Shape: (1, 75, 75, 3)
    Example: 2021-10-16-125217_5
      Frames Processed (7):
        1. Tensor Shape: (1, 75, 75, 3)
        2. Tensor Shape: (1, 75, 75, 3)
        3. Tensor Shape: (1, 75, 75, 3)
        4. Tensor Shape: (1, 75, 75, 3)
        5. Tensor Shape: (1, 75, 75, 3)
        6. Tensor Shape: (1, 75, 75, 3)
        7. Tensor Shape: (1, 75, 75, 3)
    Example: 2021-10-16-125229_9
      Frames Processed (7):
        1. Tensor Shape: (1, 75, 75, 3)
        2. Tensor Shape: (1, 75, 75, 3)
        3. Tensor Shape: (1, 75, 75, 3)
        4. Tensor Shape: (1, 75, 75, 3)
        5. Tensor Shape: (1, 75, 75, 3)
        6. Tensor Shape: (1, 75, 75, 3)
        7. Tensor Shape: (1, 75, 75, 3)
    Example: 2021-10-

In [17]:
total_frames = 0
for split in dataset_dict:
    for class_label in dataset_dict[split]:
        for example_id, frames in dataset_dict[split][class_label].items():
            total_frames += len(frames)

print(f"Total number of frames in the dataset: {total_frames}")


Total number of frames in the dataset: 10460


In [19]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

from tqdm import tqdm

# Define input shape
INPUT_SHAPE = (75, 75, 3)

# Load pre-trained InceptionV3 without the top classification layers
base_model = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=INPUT_SHAPE,
  )

# Extract features from the last convolutional layer
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# Extract features
# features = feature_extractor.predict(image_tensor)
# print("Feature shape:", features.shape)

# Initialize dictionary to store extracted features
extracted_features = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# Count total frames for tqdm progress bar
total_frames = sum(
    len(frames) for split in preprocessed_data.values()
    for classes in split.values()
    for examples in classes.values()
    for frames in examples
)

# Extract features with progress tracking
with tqdm(total=total_frames, desc="Extracting features") as pbar:
    for split, classes in preprocessed_data.items():
        for class_label, examples in classes.items():
            for example, frames in examples.items():

                # Convert list of frames into a NumPy array for batch processing
                frame_batch = np.array(frames)  # Shape: (num_frames, 75, 75, 3)

                # Extract features (InceptionV3 expects float32 input)
                frame_batch = np.squeeze(frame_batch.astype(np.float32), axis = 1)  # Ensure correct dtype
                extracted = feature_extractor.predict(frame_batch)  # Shape: (num_frames, 1, 1, 2048)

                # Store extracted features
                extracted_features[split][class_label][example] = extracted

                pbar.update(len(frames))  # Update progress bar

# Check model summary
feature_extractor.summary()

Extracting features:   0%|          | 0/8351 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Extracting features:   0%|          | 5/8351 [00:03<1:38:13,  1.42it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Extracting features:   0%|          | 12/8351 [00:07<1:20:32,  1.73it/s]

...

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 75, 75, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_94 (Conv2D)  │ (None, 37, 37,    │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │         96 │ conv2d_94[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_94       │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 35, 35,    │      9,216 │ activation_94[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │         96 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_95       │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 35, 35,    │     18,432 │ activation_95[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │        192 │ conv2d_96[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_96       │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 17, 17,    │          0 │ activation_96[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 17, 17,    │      5,120 │ max_pooling2d_4[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 17, 17,    │        240 │ conv2d_97[0][0]   │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_97       │ (None, 17, 17,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 15, 15,    │    138,240 │ activation_97[0]… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        576 │ conv2d_98[0][0]   │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_98       │ (None, 15, 15,    │          0 │ batch_normalizat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)